In [29]:
# Libraries
import pandas as pd
import re
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict

In [2]:
with open('activities.json', 'r') as file:
    activities = json.load(file)

In [5]:
#Removing words in ()
def remove_words_in_parentheses(activity):
    return re.sub(r'\([^)]*\)', '', activity)
cleaned_patterns = {remove_words_in_parentheses(activity) for activity in activities}

In [8]:
# Spliting activities and keep everything before '/'
cleaned_activities = set()
for activity in cleaned_patterns:
    if '/' in activity:
        cleaned_activity = activity.split('/')[0].strip()
    else:
        cleaned_activity = activity.strip()
    cleaned_activities.add(cleaned_activity)

In [12]:
#splitting activities and keeping everythinf before :
final_act = set()
for activity in cleaned_activities:
    if ':' in activity:
        cleaned_activity = activity.split(':')[0].strip()
    else:
        cleaned_activity = activity.strip()
    final_act.add(cleaned_activity)

In [15]:
#Converting to lowercase
final_act = {activity.lower() for activity in final_act}

In [20]:
#Lemmitizing activities
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))  # assuming English stopwords

    # Tokenize the text into words
    tokens = word_tokenize(text.lower())  # lowercase tokens
    
    # Remove stopwords and punctuation
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]

    # Lemmatize tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join tokens back into a cleaned text string
    cleaned_text = ' '.join(lemmatized_tokens)
    return cleaned_text

In [27]:
cleaned_texts = [lemmatize_text(text) for text in final_act]

In [30]:
# Finding anagrams and keeping one instance of anagrams
def keep_one_anagram(activities):
    anagram_map = defaultdict(list)

    # Create a dictionary where keys are sorted activities and values are the original activities
    for activity in activities:
        sorted_activity = ' '.join(sorted(activity.split()))
        anagram_map[sorted_activity].append(activity)

    # List to store unique activities (one from each set of anagrams)
    unique_activities = []

    # Keep one instance of each set of anagrams
    for sorted_activity, orig_activities in anagram_map.items():
        unique_activities.append(orig_activities[0])  

    return unique_activities

In [31]:
unique_activity = keep_one_anagram(cleaned_texts)

In [34]:
json_file_path = 'lemmed_activities.json'

# Save to JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(unique_activity, json_file, indent=4)